# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846802295622                   -0.70    4.8
  2   -7.852316721589       -2.26       -1.53    1.0
  3   -7.852646222388       -3.48       -2.52    3.2
  4   -7.852646677808       -6.34       -3.36    2.0
  5   -7.852646686006       -8.09       -4.72    1.5
  6   -7.852646686725       -9.14       -5.26    3.5
  7   -7.852646686730      -11.31       -6.06    1.2
  8   -7.852646686730      -12.52       -7.48    2.8
  9   -7.852646686730   +    -Inf       -7.57    2.5
 10   -7.852646686730   +  -14.75       -8.95    2.0
 11   -7.852646686730   +  -14.75       -9.62    2.0
 12   -7.852646686730      -14.75      -10.17    1.8
 13   -7.852646686730      -14.75      -10.88    2.0
 14   -7.852646686730      -14.45      -11.70    2.5
 15   -7.852646686730   +  -14.75      -12.49    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846835439666                   -0.70           4.5
  2   -7.852542307458       -2.24       -1.63   0.80    2.5
  3   -7.852638322977       -4.02       -2.72   0.80    1.0
  4   -7.852646564928       -5.08       -3.34   0.80    1.8
  5   -7.852646674969       -6.96       -4.25   0.80    1.0
  6   -7.852646686586       -7.93       -4.71   0.80    2.2
  7   -7.852646686714       -9.89       -5.49   0.80    1.0
  8   -7.852646686729      -10.82       -6.16   0.80    1.8
  9   -7.852646686730      -12.15       -7.44   0.80    1.8
 10   -7.852646686730      -13.67       -7.75   0.80    2.5
 11   -7.852646686730      -15.05       -8.81   0.80    1.0
 12   -7.852646686730   +  -15.05       -9.52   0.80    2.2
 13   -7.852646686730      -15.05      -10.73   0.80    2.0
 14   -7.852646686730   +  -15.05      -11.58   0.80    2.5
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.425051e+01     3.629710e+00
 * time: 0.46010708808898926
     1     1.660319e+00     1.896408e+00
 * time: 0.6611418724060059
     2    -1.496235e+00     2.247766e+00
 * time: 0.6880168914794922
     3    -3.409737e+00     1.962628e+00
 * time: 0.7264308929443359
     4    -4.737250e+00     1.823925e+00
 * time: 0.7644550800323486
     5    -6.493361e+00     1.077604e+00
 * time: 0.8028309345245361
     6    -7.311022e+00     7.195130e-01
 * time: 0.8410749435424805
     7    -7.608226e+00     4.723921e-01
 * time: 0.8673088550567627
     8    -7.759735e+00     1.166974e-01
 * time: 0.8939299583435059
     9    -7.801999e+00     1.047776e-01
 * time: 0.9208180904388428
    10    -7.824707e+00     7.315319e-02
 * time: 0.9470019340515137
    11    -7.836876e+00     4.788541e-02
 * time: 0.9734790325164795
    12    -7.843946e+00     4.454044e-02
 * time: 0.9998219013214111
    13    -7.844323e+00     5.029710e-02
 * time: 1.026576995

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846864240493                   -0.70    4.8
  2   -7.852322128278       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686713                   -2.54
  2   -7.852646686730      -10.78       -6.04
  3   -7.852646686730   +  -14.75      -12.66


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.7664626035878636e-13
|ρ_newton - ρ_scfv| = 2.0341119291698525e-13
|ρ_newton - ρ_dm|   = 1.0004391886507098e-9
